In [ ]:
import os
print('Get current working directory : ', os.getcwd())
os.chdir('c:\\Users\\celine\\Desktop')
data = pd.read_csv('all_games.csv')
data.columns

In [ ]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm
from time import sleep

In [ ]:
from selenium import webdriver
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36")
driver = webdriver.Firefox(profile)
driver.get('https://www.jeuxvideo.com/jeux/jeu-1571827/')
button = driver.find_elements(By.CSS_SELECTOR, "button")
button[1].click()

def close_interstitial():
    try:
        driver.find_element(By.CLASS_NAME,'interstitial-top-close').click()
    except Exception:
        sleep(10)     

    
for i, row in tqdm(data.iterrows(), total=len(data)):
    if not pd.isnull(row.Classification):
        continue
    try:
        driver.find_element(By.CLASS_NAME,'icon-search').click()
    except Exception:
        sleep(5)
        close_interstitial()
        driver.find_element(By.CLASS_NAME,'icon-search').click()    
    barre=driver.find_element(By.NAME,'q')
    barre.clear()
    barre.send_keys(row['name'])
    barre.send_keys(Keys.ENTER)
    sleep(3)
    try: 
        driver.find_element(By.CSS_SELECTOR, '.dfpFooter__button').click()
    except Exception:
        pass 
    
    try:    
        driver.find_element(By.CSS_SELECTOR, 'a.card__link').click()
    except Exception:
        try:
            sleep(5) 
            close_interstitial()   
            driver.find_element(By.CSS_SELECTOR, 'a.card__link').click()
        except Exception:
            continue
    sleep(3)    
    try:
        classification=(driver.find_element(By.XPATH, "//span[contains(text(), ' ans')]")).text
    except Exception:
        classification='NA'
    #print(classification)
    data.loc[i,"Classification"]=classification
driver.close()

In [ ]:
data.to_csv('all_games.csv', sep = ',') 